In [1]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup as bts
import pandas as pd
import re
import time
import sys
from datetime import date, datetime

In [2]:
def getAndParseURL(url):
    result = requests.get(url, headers={"User-Agent":"Mozilla/5.0"})
    soup = bts(result.text, 'html.parser')
    return soup

In [3]:
url = "https://www.imdb.com/chart/top"
soup = getAndParseURL(url)
movie_urls = []
for i in soup.tbody.findAll('tr'):
    movie_urls.append('https://www.imdb.com'+i.findAll('td')[1].a['href'])

In [4]:
row = []
count = 0
for i in movie_urls:
    
    count += 1
    html = getAndParseURL(i)

    try:
        title = html.find('h1', {"data-testid":"hero-title-block__title"}).text
    except:
        title = None
        print("Couldn't scrape the title data")

    try:
#         revenue = html.find('li', {"data-testid":"title-boxoffice-cumulativeworldwidegross"}).findAll('span')[1].text
        revenue = int(re.findall("\d+", ''.join(html.find('li', {"data-testid":"title-boxoffice-cumulativeworldwidegross"}).findAll('span')[1].text.split(',')))[0])
    except:
        revenue = None
        print("Couldn't scrape the revenue data")

    try:
#         budget = html.find('li', {"data-testid":"title-boxoffice-budget"}).findAll('span')[1].text.split(' ')[0]
        budget = int(re.findall("\d+", ''.join(html.find('li', {"data-testid":"title-boxoffice-budget"}).li.text.split(',')))[0])
    except:
        budget = None
        print("Couldn't scrape the budget data")
        
    try:
        runtime = html.find('ul', {"data-testid":"hero-title-block__metadata"}).findAll('li')[2].text
    except:
        runtime = None
        print("Couldn't scrape the runtime data")
        
    try:
        certificate = html.find('ul', {"data-testid":"hero-title-block__metadata"}).findAll('li')[1].span.text
    except:
        certificate = None
        print("Couldn't scrape the certificate data")

    try:
        releaseDate = pd.to_datetime(html.find('li', {"data-testid":"title-details-releasedate"}).li.a.text.split('(')[0].strip())
    except:
        releaseDate = None
        print("Couldn't scrape the date data")

    try:
        score = int(10*float(html.find('div', {"data-testid":"hero-rating-bar__aggregate-rating__score"}).text.split('/')[0]))
    except:
        score = None
        print("Couldn't scrape the score data")

    try:
        votes = html.find('div', {"data-testid":"hero-rating-bar__aggregate-rating"}).a.div.div.findAll('div')[1].findAll('div')[2].text
        if 'M' in votes:
            votes = int(float(votes.split('M')[0])*1_000_000)
        elif 'K' in votes:
            votes = int(float(votes.split('K')[0])*1_000)
        else:
            votes = int(votes)
    except:
        votes = None
        print("Couldn't scrape the vote data")

    try:
        directors = []
        for i in html.find('div', {"data-testid":"title-pc-wide-screen"}).findAll('li', {"data-testid":"title-pc-principal-credit"})[0].findAll('a'):
            directors.append(i.text)
    except:
        directors = None
        print("Couldn't scrape the directors data")

    try:
        writers = []
        for i in html.find('div', {"data-testid":"title-pc-wide-screen"}).findAll('li', {"data-testid":"title-pc-principal-credit"})[1].findAll('a'):
            writers.append(i.text)
    except:
        writers = None
        print("Couldn't scrape the writers data")

    try:
        stars = []
        for i in html.find('div', {"data-testid":"title-pc-wide-screen"}).findAll('li', {"data-testid":"title-pc-principal-credit"})[2].find('ul', {"role":"presentation"}).findAll('a'):
            stars.append(i.text)
    except:
        stars = None
        print("Couldn't scrape the stars data")

    try:
        genres = []
        for i in html.find('li', {"data-testid":"storyline-genres"}).findAll('li'):
            genres.append(i.text)
    except:
        genres = None
        print("Couldn't scrape the genres data")

    row.append([title, revenue, budget, runtime, certificate, releaseDate,
                score, votes, directors, writers, stars, genres])
    
    print(count, ": ", title, "...Done✔", sep="")
    print("Sleeping 5 seconds...")
    time.sleep(5)

column = ["Title", "Revenues", "Budget", "Runtime", "Certificate", "ReleaseDate",
          "Score", "Votes", "Directors", "Writers", "Stars", "Genres"]
dfMovies = pd.DataFrame.from_records(row, columns=column)

1: The Shawshank Redemption...Done✔
Sleeping 5 seconds...
2: The Godfather...Done✔
Sleeping 5 seconds...
3: The Godfather: Part II...Done✔
Sleeping 5 seconds...
4: The Dark Knight...Done✔
Sleeping 5 seconds...
5: 12 Angry Men...Done✔
Sleeping 5 seconds...
6: Schindler's List...Done✔
Sleeping 5 seconds...
7: The Lord of the Rings: The Return of the King...Done✔
Sleeping 5 seconds...
8: Pulp Fiction...Done✔
Sleeping 5 seconds...
9: Il buono, il brutto, il cattivo...Done✔
Sleeping 5 seconds...
10: The Lord of the Rings: The Fellowship of the Ring...Done✔
Sleeping 5 seconds...
11: Fight Club...Done✔
Sleeping 5 seconds...
12: Forrest Gump...Done✔
Sleeping 5 seconds...
13: Inception...Done✔
Sleeping 5 seconds...
14: The Lord of the Rings: The Two Towers...Done✔
Sleeping 5 seconds...
15: Star Wars: Episode V - The Empire Strikes Back...Done✔
Sleeping 5 seconds...
16: The Matrix...Done✔
Sleeping 5 seconds...
17: Goodfellas...Done✔
Sleeping 5 seconds...
18: One Flew Over the Cuckoo's Nest...Don

In [7]:
dfMovies.to_csv("data/movies.csv")
dfMovies

,Title,Revenues,Budget,Runtime,Certificate,ReleaseDate,Score,Votes,Directors,Writers,Stars,Genres
0,The Shawshank Redemption,2.881729e+07,25000000.0,2h 22m,13+,1995-03-10,93,2500000,[Frank Darabont],"[Stephen King, Frank Darabont]","[Tim Robbins, Morgan Freeman, Bob Gunton]",[Drama]
1,The Godfather,2.461210e+08,6000000.0,2h 55m,18+,1973-10-01,92,1700000,[Francis Ford Coppola],"[Mario Puzo, Francis Ford Coppola]","[Marlon Brando, Al Pacino, James Caan]","[Crime, Drama]"
2,The Godfather: Part II,4.803578e+07,13000000.0,3h 22m,18+,1974-12-18,90,1200000,[Francis Ford Coppola],"[Francis Ford Coppola, Mario Puzo]","[Al Pacino, Robert De Niro, Robert Duvall]","[Crime, Drama]"
3,The Dark Knight,1.005974e+09,185000000.0,2h 32m,16+,2008-07-25,90,2500000,[Christopher Nolan],"[Writers, Jonathan Nolan, Christopher Nolan, D...","[Christian Bale, Heath Ledger, Aaron Eckhart]","[Action, Crime, Drama, Thriller]"
4,12 Angry Men,9.550000e+02,350000.0,1h 36m,15+,1960-04-01,90,740000,[Sidney Lumet],[Reginald Rose],"[Henry Fonda, Lee J. Cobb, Martin Balsam]","[Crime, Drama]"
...,...,...,...,...,...,...,...,...,...,...,...,...
245,Le notti di Cabiria,7.553190e+05,NaN,1h 50m,Not Rated,1957-05-10,81,47000,[Federico Fellini],"[Writers, Federico Fellini, Ennio Flaiano, Tul...","[Giulietta Masina, François Périer, Franca Marzi]",[Drama]
246,The Princess Bride,3.090244e+07,16000000.0,1h 38m,PG,1987-09-25,81,413000,[Rob Reiner],[William Goldman],"[Cary Elwes, Mandy Patinkin, Robin Wright]","[Adventure, Family, Fantasy, Romance]"
247,"Paris, Texas",2.207548e+06,1162000.0,2h 25m,13A,2020-10-02,81,101000,[Wim Wenders],"[L.M. Kit Carson, Sam Shepard, Walter Donohue]","[Harry Dean Stanton, Nastassja Kinski, Dean St...",[Drama]
248,Trois couleurs: Rouge,3.581969e+06,NaN,1h 39m,15+,1995-02-03,81,99000,[Krzysztof Kieslowski],"[Writers, Krzysztof Kieslowski, Krzysztof Pies...","[Irène Jacob, Jean-Louis Trintignant, Frédériq...","[Drama, Mystery, Romance]"


In [8]:
dfMovies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Title        250 non-null    object        
 1   Revenues     241 non-null    float64       
 2   Budget       221 non-null    float64       
 3   Runtime      250 non-null    object        
 4   Certificate  250 non-null    object        
 5   ReleaseDate  250 non-null    datetime64[ns]
 6   Score        250 non-null    int64         
 7   Votes        250 non-null    int64         
 8   Directors    250 non-null    object        
 9   Writers      250 non-null    object        
 10  Stars        250 non-null    object        
 11  Genres       250 non-null    object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(7)
memory usage: 23.6+ KB
